In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import json, sys, random
import numpy as np

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import adam
import keras.callbacks

In [3]:
from PIL import Image, ImageDraw 
from matplotlib import pyplot as plt

In [4]:
f = open(r'../input/ships-in-satellite-imagery/shipsnet.json')
dataset = json.load(f)
f.close()

In [5]:
input_data = np.array(dataset['data']).astype('uint8')
output_data = np.array(dataset['labels']).astype('uint8')

In [6]:
input_data.shape

(4000, 19200)

In [7]:
n_spectrum = 3 # color chanel (RGB)
weight = 80
height = 80
X = input_data.reshape([-1, n_spectrum, weight, height])
X[0].shape

(3, 80, 80)

In [10]:
output_data.shape

(4000,)

In [11]:
output_data

array([1, 1, 1, ..., 0, 0, 0], dtype=uint8)

In [12]:
# output encoding
y = np_utils.to_categorical(output_data, 2)

In [14]:
# shuffle all indexes
indexes = np.arange(4000)
np.random.shuffle(indexes)

In [15]:
X_train = X[indexes].transpose([0,2,3,1])
y_train = y[indexes]

In [16]:
# normalization
X_train = X_train / 255

In [17]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(80, 80, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) #40x40
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) #20x20
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) #10x10
model.add(Dropout(0.25))

model.add(Conv2D(32, (10, 10), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) #5x5
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

In [21]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [22]:
model.fit(
    X_train, 
    y_train,
    batch_size=32,
    epochs=10,
    validation_split=0.2,
    shuffle=True,
    verbose=2)

Train on 3200 samples, validate on 800 samples
Epoch 1/10
 - 22s - loss: 0.4009 - accuracy: 0.8269 - val_loss: 0.2245 - val_accuracy: 0.9200
Epoch 2/10
 - 20s - loss: 0.2022 - accuracy: 0.9134 - val_loss: 0.2281 - val_accuracy: 0.9062
Epoch 3/10
 - 21s - loss: 0.1329 - accuracy: 0.9509 - val_loss: 0.0796 - val_accuracy: 0.9825
Epoch 4/10
 - 20s - loss: 0.1153 - accuracy: 0.9600 - val_loss: 0.0894 - val_accuracy: 0.9825
Epoch 5/10
 - 20s - loss: 0.0940 - accuracy: 0.9666 - val_loss: 0.0695 - val_accuracy: 0.9737
Epoch 6/10
 - 20s - loss: 0.0749 - accuracy: 0.9731 - val_loss: 0.1041 - val_accuracy: 0.9650
Epoch 7/10
 - 20s - loss: 0.1022 - accuracy: 0.9641 - val_loss: 0.0568 - val_accuracy: 0.9862
Epoch 8/10
 - 20s - loss: 0.0524 - accuracy: 0.9803 - val_loss: 0.0264 - val_accuracy: 0.9925
Epoch 9/10
 - 20s - loss: 0.0477 - accuracy: 0.9822 - val_loss: 0.0614 - val_accuracy: 0.9825
Epoch 10/10
 - 20s - loss: 0.0498 - accuracy: 0.9816 - val_loss: 0.0356 - val_accuracy: 0.9887
